In [7]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
import boto3
from nba_api.stats.endpoints import LeagueGameFinder, LeagueDashPlayerBioStats
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import sys
import allinone

pd.set_option('display.max_columns', 500)

# Example usage
db_name = "thebigone"
username = "ajwin"
password = "CharlesBark!23"
host = "nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com"


['C:\\Users\\ajwin\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\ajwin\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\ajwin\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\ajwin\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\ajwin\\Projects\\NBA\\thebigone\\.venv', '', 'c:\\Users\\ajwin\\Projects\\NBA\\thebigone\\.venv\\Lib\\site-packages', 'c:\\Users\\ajwin\\Projects\\NBA\\thebigone\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ajwin\\Projects\\NBA\\thebigone\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ajwin\\Projects\\NBA\\thebigone\\.venv\\Lib\\site-packages\\Pythonwin']


In [3]:
# CREATE DATABASE

# create_rds_instance("nba-rds-instance","thebigone","ajwin","CharlesBark!23")


RDS instance nba-rds-instance creation initiated.


{'DBInstance': {'DBInstanceIdentifier': 'nba-rds-instance',
  'DBInstanceClass': 'db.t3.micro',
  'Engine': 'postgres',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'ajwin',
  'DBName': 'thebigone',
  'AllocatedStorage': 20,
  'PreferredBackupWindow': '06:40-07:10',
  'BackupRetentionPeriod': 7,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-048e804df880ce6ca',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'default',
   'DBSubnetGroupDescription': 'default',
   'VpcId': 'vpc-0da11b2b953a7cab5',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-02fedebb493b9a06e',
     'SubnetAvailabilityZone': {'Name': 'us-east-1c'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-0c8f0f9dc389265c1',
     'SubnetAvailabilityZone': {'Name': 'us-east-1e'},
     'S

In [4]:


# Create a table in PostgreSQL

# Data type mapping from pandas to PostgreSQL


conn = dbfuncs.connect_to_rds(db_name, username, password, host)


Connected to RDS database


In [7]:

dbfuncs.delete_table(conn,table_name="players")
dbfuncs.delete_table(conn,table_name="games")


# Get and insert games data
games_data = dbfuncs.get_games_data()
dbfuncs.create_table(conn, "games", games_data)
dbfuncs.insert_dataframe(conn, "games", games_data)

# Get and insert players data
players_data = dbfuncs.get_players_data()
dbfuncs.create_table(conn, "players", players_data)
dbfuncs.insert_dataframe(conn, "players", players_data)


Table games created successfully with inferred schema.
10815 records inserted into games successfully.
Table players created successfully with inferred schema.
2256 records inserted into players successfully.


In [5]:

# Function to delete a table from the PostgreSQL database


Table players deleted successfully.
Table games deleted successfully.


In [23]:
#############################################################
#############################################################

def connect_to_rds(db_name, username, password, host, port=5432):
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=username,
            password=password,
            host=host,
            port=port
        )
        print("Connected to RDS database")
        return conn
    except Exception as e:
        print(f"Error connecting to RDS: {str(e)}")
        return None
    
# Query the database and return results as a pandas dataframe
def query_database_to_dataframe(conn, query):
    try:
        # Read the query result into a pandas dataframe
        dataframe = pd.read_sql(query, conn)
        return dataframe
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        return None

# Connect to the RDS database
conn = connect_to_rds(db_name, username, password, host)

# Define your SQL query
query = "SELECT * FROM playerids;"  # Example query: fetch 10 rows from the 'games' table

# Execute the query and store the result in a pandas dataframe
if conn:
    result_df = query_database_to_dataframe(conn, query)


# Close the connection
conn.close()

Connected to RDS database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_84252\3665034828.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


In [24]:
result_df

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
15052,78647,Jim Zoet,Jim,Zoet,False
15053,78648,Bill Zopf,Bill,Zopf,False
15054,1627826,Ivica Zubac,Ivica,Zubac,True
15055,78650,Matt Zunic,Matt,Zunic,False
